In [1]:
import pandas as pd
import datetime
import numpy as np
import pandas as pd
import datetime

In [2]:
df1 = pd.read_csv('base_de_dados/dados_limpos.csv', sep=';')
df1['dataCaso'] = pd.to_datetime(df1['dataCaso'])
# data['dataCaso'] = data['dataCaso'].dt.date

A:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420655 entries, 0 to 1420654
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   Unnamed: 0             1420655 non-null  int64         
 1   identificadorCaso      1420655 non-null  object        
 2   profissionalSaudeEsus  620023 non-null   object        
 3   bairroCaso             1420655 non-null  object        
 4   sexoCaso               1420655 non-null  object        
 5   idadeCaso              1418880 non-null  float64       
 6   faixaEtaria            1420655 non-null  object        
 7   resultadoFinalExame    1420655 non-null  object        
 8   obitoConfirmado        1420655 non-null  object        
 9   dataCaso               1420655 non-null  datetime64[ns]
 10  profissoes             38422 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 119.2+ MB


In [4]:
df1.faixaEtaria.value_counts().index.to_list()

['35 a 39 anos',
 '30 a 34 anos',
 '25 a 29 anos',
 '40 a 44 anos',
 '20 a 24 anos',
 '45 a 49 anos',
 '50 a 54 anos',
 '55 a 59 anos',
 '60 a 64 anos',
 '15 a 19 anos',
 '65 a 69 anos',
 '00 a 04 anos',
 '70 a 74 anos',
 '80 ou mais',
 '10 a 14 anos',
 '05 a 09 anos',
 '75 a 79 anos',
 'Sem Informacao']

# COISAS QUE QUERO TRANSFORMAR EM NUMERICA:
- Numero de obitos (FOI)
- Numero de casos positivos (FOI)
- Media de idade por dia (FOI)
- Obitos e casos por sexo (FOI)
- Obitos e casos por raça por faixa de idade

# ==============================================================================

# Refatorando faixa etaria

In [5]:
df1.faixaEtaria.replace('00 a 04 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('05 a 09 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('10 a 14 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('15 a 19 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('20 a 24 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('25 a 29 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('30 a 34 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('35 a 39 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('40 a 44 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('45 a 49 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('50 a 54 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('55 a 59 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('60 a 64 anos', '60 a 79 anos', inplace=True)
df1.faixaEtaria.replace('65 a 69 anos', '60 a 79 anos', inplace=True)
df1.faixaEtaria.replace('70 a 74 anos', '60 a 79 anos', inplace=True)
df1.faixaEtaria.replace('75 a 79 anos', '60 a 79 anos', inplace=True)

# ==============================================================================

# Numero de casos confirmados

In [6]:
filtroConfirmado = (df1.resultadoFinalExame == 'Positivo')
dfConfirmado = df1[filtroConfirmado]

grouped = dfConfirmado.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria'])['resultadoFinalExame'].count().reset_index().sort_values('dataCaso')

num_confirmado = {'data': grouped.dataCaso.to_list(), 'bairro': grouped.bairroCaso.to_list(), 'faixa Etaria': grouped.faixaEtaria.to_list(), 'casos confirmados': grouped.resultadoFinalExame.to_list()}
num_confirmado = pd.DataFrame(num_confirmado)

In [7]:
num_confirmado.set_index(['data', 'bairro', 'faixa Etaria'], inplace=True)
num_confirmado.head()

casos confirmados
data       bairro        faixa Etaria                   
2020-01-01 ALDEOTA       20 a 39 anos                  3
           LAGOA REDONDA 20 a 39 anos                  1
           MARAPONGA     40 a 59 anos                  1
           MEIRELES      40 a 59 anos                  1
           MESSEJANA     20 a 39 anos                  1

# ==============================================================================

# Numero de obitos

In [8]:
filtroObito = (df1.obitoConfirmado == 'Verdadeiro')
dfObito = df1[filtroObito]

grouped = dfObito.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria'])['obitoConfirmado'].count().reset_index().sort_values('dataCaso')

num_obitos = {'data': grouped.dataCaso.to_list(), 'bairro': grouped.bairroCaso.to_list(), 'faixa Etaria': grouped.faixaEtaria.to_list(), 'obitos': grouped.obitoConfirmado.to_list()}
num_obitos = pd.DataFrame(num_obitos)

In [9]:
num_obitos.set_index(['data', 'bairro', 'faixa Etaria'], inplace=True)
num_obitos.head()

obitos
data       bairro         faixa Etaria        
2020-01-01 Indeterminado  40 a 59 anos       1
           JANGURUSSU     60 a 79 anos       1
2020-01-04 VILA VELHA     40 a 59 anos       1
2020-01-05 QUINTINO CUNHA 20 a 39 anos       1
2020-01-06 FATIMA         40 a 59 anos       1

# ==============================================================================

# Numeros segmentados pelo sexo

## Confirmados

In [10]:
grouped = dfConfirmado.groupby(['sexoCaso', 'bairroCaso', 'faixaEtaria', pd.Grouper(key='dataCaso', freq='D')])['resultadoFinalExame'].count().reset_index().sort_values('dataCaso')

In [11]:
grouped = pd.get_dummies(grouped, columns=['sexoCaso'])
grouped['confirmados_masculinos'] = np.where(grouped['sexoCaso_MASCULINO'] == 1, grouped['resultadoFinalExame'], 0)
grouped['confirmados_feminino'] = np.where(grouped['sexoCaso_FEMININO'] == 1, grouped['resultadoFinalExame'], 0)
grouped.drop(columns=['resultadoFinalExame', 'sexoCaso_FEMININO', 'sexoCaso_MASCULINO'], inplace=True)

In [12]:
aggregation_functions = {'confirmados_masculinos': 'sum', 'confirmados_feminino': 'sum'}
num_confirmado_sexo = grouped.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria']).aggregate(aggregation_functions)

In [13]:
num_confirmado_sexo.head()

confirmados_masculinos  \
dataCaso   bairroCaso      faixaEtaria                            
2020-01-01 ALDEOTA         20 a 39 anos                       1   
                           40 a 59 anos                       0   
                           60 a 79 anos                       0   
           ALTO DA BALANÇA 20 a 39 anos                       0   
                           60 a 79 anos                       0   

                                         confirmados_feminino  
dataCaso   bairroCaso      faixaEtaria                         
2020-01-01 ALDEOTA         20 a 39 anos                     2  
                           40 a 59 anos                     1  
                           60 a 79 anos                     1  
           ALTO DA BALANÇA 20 a 39 anos                     1  
                           60 a 79 anos                     1

## Obitos

In [14]:
grouped = dfObito.groupby(['sexoCaso', 'bairroCaso', 'faixaEtaria', pd.Grouper(key='dataCaso', freq='D')])['obitoConfirmado'].count().reset_index().sort_values('dataCaso')

In [15]:
grouped = pd.get_dummies(grouped, columns=['sexoCaso'])
grouped['obitos_masculinos'] = np.where(grouped['sexoCaso_MASCULINO'] == 1, grouped['obitoConfirmado'], 0)
grouped['obitos_feminino'] = np.where(grouped['sexoCaso_FEMININO'] == 1, grouped['obitoConfirmado'], 0)
grouped.drop(columns=['obitoConfirmado', 'sexoCaso_FEMININO', 'sexoCaso_MASCULINO'], inplace=True)

In [16]:
aggregation_functions = {'obitos_masculinos': 'sum', 'obitos_feminino': 'sum'}
num_obitos_sexo = grouped.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria']).aggregate(aggregation_functions)

In [17]:
num_obitos_sexo.head()

obitos_masculinos  obitos_feminino
dataCaso   bairroCaso     faixaEtaria                                     
2020-01-01 Indeterminado  40 a 59 anos                  1                0
           JANGURUSSU     60 a 79 anos                  1                0
2020-01-04 VILA VELHA     40 a 59 anos                  1                0
2020-01-05 QUINTINO CUNHA 20 a 39 anos                  1                0
2020-01-06 FATIMA         40 a 59 anos                  1                0

# ==============================================================================

# Juntando tudo

In [18]:
base = pd.concat([num_confirmado, num_obitos, num_confirmado_sexo, num_obitos_sexo], axis=1)
base.fillna(0, inplace=True)
base.head()

casos confirmados  obitos  \
2020-01-01 ALDEOTA         20 a 39 anos                  3     0.0   
                           40 a 59 anos                  1     0.0   
                           60 a 79 anos                  1     0.0   
           ALTO DA BALANÇA 20 a 39 anos                  1     0.0   
                           60 a 79 anos                  1     0.0   

                                         confirmados_masculinos  \
2020-01-01 ALDEOTA         20 a 39 anos                       1   
                           40 a 59 anos                       0   
                           60 a 79 anos                       0   
           ALTO DA BALANÇA 20 a 39 anos                       0   
                           60 a 79 anos                       0   

                                         confirmados_feminino  \
2020-01-01 ALDEOTA         20 a 39 anos                     2   
                           40 a 59 anos                     1   
                           60 a 79 anos                     1   
           ALTO DA BALANÇA 20 a 39 anos                     1   
                           60 a 79 anos                     1   

                                         obitos_masculinos  obitos_feminino  
2020-01-01 ALDEOTA         20 a 39 anos                0.0              0.0  
                           40 a 59 anos                0.0              0.0  
                           60 a 79 anos                0.0              0.0  
           ALTO DA BALANÇA 20 a 39 anos                0.0              0.0  
                           60 a 79 anos                0.0              0.0

In [19]:
base.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 84274 entries, (Timestamp('2020-01-01 00:00:00'), 'ALDEOTA', '20 a 39 anos') to (Timestamp('2021-08-12 00:00:00'), 'Indeterminado', '00 a 19 anos')
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   casos confirmados       84274 non-null  int64  
 1   obitos                  84274 non-null  float64
 2   confirmados_masculinos  84274 non-null  int64  
 3   confirmados_feminino    84274 non-null  int64  
 4   obitos_masculinos       84274 non-null  float64
 5   obitos_feminino         84274 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 4.2+ MB


# ==============================================================================

# Obitos e casos normalizados

In [20]:
base['casos_confirmados_normalizado'] = base['casos confirmados'] / max(base['casos confirmados'].to_list())

In [21]:
base['obitos_normalizado'] = base['obitos'] / max(base['obitos'].to_list())

# ==============================================================================

In [22]:
base.to_csv('base_de_dados/Base_de_dados.csv', sep=';')